![University of Tehran](./img/UT.png)
#   <font color='red'><center>AI CA 3<center></font> 
## <center>Dr. Fadaei<center>
### <center>Daniyal Maroufi<center>
### <center>810098039<center>


## Aim

This assignment aims to use Naive Bayes networks to build a classifier model to predict the category of an article from Digikala using its excerpt.


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from __future__ import unicode_literals
from hazm import *


# Phase 1 - Data Preprocessing



In Natural Language Processing, it is agreed that using the root of the words is better for classification accuracy. The prefixes and postfixes of the words are not that necessary to be in sequence, and sometimes they may even inversely affect the accuracy because all forms of a word have the same meaning. For this purpose, there are two methods, Stemming and Lemmatization. Stemming removes the most common prefixes and postfixes of a word to find the root, while, Lemmatization uses an entire dictionary and finds the actual root of the words.